In [4]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from imblearn.over_sampling import SMOTE
from nltk import bigrams
from sklearn.pipeline import Pipeline

In [2]:
# txt_5 = nltk.word_tokenize(' '.join(df_5['reviewText']))
# text_dict = Counter(txt_5)
# adj_5 = {word:count for word,count in text_dict.items() if is_adjective(word)}

In [2]:
file = ('C:\\Users\\Abe\\Data Science Bootcamp\\Unit 3\\Potpourrie\\Amazon\\Sports_and_Outdoors_5.json')
df = pd.read_json(file, lines=True)

df['sentiment'] = df['overall'].isin([4,5])

In [6]:
# Create column with adjectives only

df['adjective'] = df['reviewText'].apply(nltk.word_tokenize)
df['adjective'] = df['adjective'].apply(nltk.pos_tag)
df['adjective'] = pd.Series([[word for (word,pos) in item if pos == 'JJ'] for item in df['adjective']])
df['adjective'] = df['adjective'].apply(' '.join)
df.head()

In [3]:
df_train = df.sample(frac=.8)
df_test = df.drop(df_train.index)

keep = ['overall','reviewText','summary','sentiment']
df_train = df_train[keep]
df_test = df_test[keep]

In [4]:
# Undersampling

rating_1_count = df_train[df_train['overall'] == 1].count()[0]

df_ss = pd.DataFrame()
for num in [1,2,3,4,5]:
    df_ss = df_ss.append(df_train[df_train['overall'] == num].sample(n=rating_1_count),ignore_index=True)

In [5]:
review_dict = {}
for name, group in df_train.groupby('overall'):
    review_dict[name] = nltk.word_tokenize(' '.join(group['summary']))

In [6]:
count_dict = {}
for key,value in review_dict.items():
    count_dict[key] = dict(Counter(value))

In [7]:
def is_adjective(word):
    return nltk.pos_tag([word])[0][1] == 'JJ'

final_dict = {}
for rating in count_dict:
    adj_dict = {}
    for word,count in count_dict[rating].items():
        if is_adjective(word):
            adj_dict[word] = count
    final_dict[rating] = adj_dict

In [8]:
df_adj = pd.DataFrame([(k,k1,v1) for k,v in final_dict.items() for k1,v1 in v.items()],
                      columns=['rating','adjective','count'])

In [9]:
df_train.groupby('overall').count()

,reviewText,summary,sentiment
overall,,,
1,7253,7253,7253
2,8131,8131,8131
3,19244,19244,19244
4,51641,51641,51641
5,150801,150801,150801


In [10]:
# for rating, subset in df_adj.groupby('rating'):
#     print(subset.sort_values('count',ascending=False)[:5])

In [11]:
# df_adj_dup = df_adj.drop_duplicates('adjective',keep=False)

# for rating,subset in df_adj_dup.groupby('rating'):
#     print(subset.sort_values('count',ascending=False)[:5])

In [10]:
# vectorizer = HashingVectorizer(stop_words='english', non_negative=True)
# X_train = vectorizer.transform(df_train['summary'])

In [5]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

nb = GaussianNB()

In [6]:
from sklearn.base import TransformerMixin

class DenseTransformer(TransformerMixin):
    def fit_transform(self, x, y=None, **fitparams):
        return self.transform(x)
    def fit(self, x, y=None, **fitparams):
        return self
    def transform(self, x, y=None, **fitparams):
        return x.toarray()

In [7]:
X_train = df_train['summary']
X_test = df_test['summary']
Y_train= df_train['sentiment']
Y_test = df_test['sentiment']

vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 3), stop_words='english')
ch2 = SelectKBest(chi2, k=200)
transformer = DenseTransformer()
nb = GaussianNB()

pipeline = Pipeline([('vectorizer',vectorizer),('Kbest',ch2),('transformer',transformer),('nb_model',nb)])
pipeline.fit(X_train,Y_train)
pipeline.score(X_test,Y_test)

0.87691295324548235

In [6]:
# Grid search parameters

for num in [200]:
    # Select X and Y
    X_train = df_train['summary']
    X_test = df_test['summary']
    Y_train= df_train['sentiment']
    Y_test = df_test['sentiment']
    
    # Turn the words into vectors
    vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 3), stop_words='english')
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # Select K Best features
    ch2 = SelectKBest(chi2, k=num)
    X_train = ch2.fit_transform(X_train, Y_train)
    X_test = ch2.transform(X_test)
    
    nb_model = nb.fit(X_train.toarray(),Y_train)
    # nb.score(X_train.toarray(),Y_train)
    
    print(num)
    print(nb.score(X_test.toarray(),Y_test))
    print(confusion_matrix(nb_model.predict(X_test.toarray()), Y_test))

200
0.877351645941
[[ 2949  1554]
 [ 5715 49049]]


In [57]:
# Oversampling using SMOTE

# sm = SMOTE()
# X_train_res, Y_train_res = sm.fit_sample(X_train.toarray(),Y_train)
# X_train_res.shape

In [11]:
Y_test.value_counts()[1] / Y_test.value_counts().sum(0)

0.85275110938633636

In [32]:
pos_review = ['Not the greatest but it works']
predict_me = vectorizer.transform(pos_review)
predict_me = ch2.transform(predict_me)

nb_model.predict(predict_me.toarray())

array([ True], dtype=bool)

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import LinearSVC, SVC



n_estimators = 20
svc = BaggingClassifier(SVC(kernel='linear', probability=True, class_weight='balanced'),
                        max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs=3)

# l_svc = LinearSVC()


pipeline = Pipeline([('vectorizer',vectorizer),('Kbest',ch2),('svc',svc)])
pipeline.fit(X_train,Y_train)
print(pipeline.score(X_test,Y_test))
confusion_matrix(pipeline.predict(X_test),Y_test)

0.873723994803


array([[ 2087,   873],
       [ 6611, 49696]])

In [ ]:
print(svc.score(X_test.toarray(),Y_test))
confusion_matrix(svc_model.predict(X_test.toarray(),Y_test))

In [34]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_features=100,
                              max_depth=30)
tree_model = tree.fit(X_train_res,Y_train_res)
tree.score(X_train_res,Y_train_res)

0.61033283607156619

In [35]:
print(tree.score(X_test.toarray(),Y_test))
confusion_matrix(tree_model.predict(X_test.toarray()), Y_test)

0.863161624513


array([[ 1852,  1270],
       [ 6840, 49305]])

In [ ]:
# Try Oversampling the bad review
# Remove nouns